# Table 3. Descriptive Outcomes
On-scene survival (Yes=1, No=0)
ICU yes (Length >0)
If ICU yes, Length of ICU stay
Length of hospital stay
Survival of hospital stay
1 y survival


In [ ]:
import pandas as pd

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/pediatric_trauma/data/Data_PedRegaTrauma_coded_for_analysis_250417.xlsx'

In [ ]:
df = pd.read_excel(data_path, sheet_name='All centres cleaned')

In [ ]:
df['length of ICU stay'].unique()

In [ ]:
def create_table3(df):    
    assert df["Pat ID"].nunique() == len(df), "There are duplicate patient IDs in the dataset." 

    pop_df = pd.DataFrame()
    str_df = pd.DataFrame()

    # get the number of patients
    n_patients = len(df)
    pop_df['n_patients'] = [n_patients]
    str_df['n_patients'] = [n_patients]

    # On-scene survival (Yes=1, No=0)
    pop_df['n_on_scene_survival'] = [len(df[df['on-sceen survival (y/n)'] == 1])]
    pop_df['prop_on_scene_survival'] = [pop_df['n_on_scene_survival'].values[0] / n_patients]
    str_df['prop_on_scene_survival'] = f'{pop_df["n_on_scene_survival"].values[0]} ({pop_df["prop_on_scene_survival"].values[0]:.1%})'

    # ICU yes (Length >0)
    df['length of ICU stay'] = df['length of ICU stay'].replace(999, pd.NA)
    pop_df['n_icu_yes'] = [len(df[df['length of ICU stay'] > 0])]
    pop_df['prop_icu_yes'] = [pop_df['n_icu_yes'].values[0] / n_patients]
    str_df['prop_icu_yes'] = f'{pop_df["n_icu_yes"].values[0]} ({pop_df["prop_icu_yes"].values[0]:.1%})'

    # If ICU yes, Length of ICU stay
    with_icu_df = df[df['length of ICU stay'] > 0]
    pop_df['icu_los_median'] = [with_icu_df['length of ICU stay'].median()]
    pop_df['icu_los_q1'] = [with_icu_df['length of ICU stay'].quantile(0.25)]
    pop_df['icu_los_q3'] = [with_icu_df['length of ICU stay'].quantile(0.75)]
    str_df['icu_los'] = f'{pop_df["icu_los_median"].values[0]} [{pop_df["icu_los_q1"].values[0]}-{pop_df["icu_los_q3"].values[0]}]'
    # Length of hospital stay
    # Survival of hospital stay
    # 1 y survival

    return pop_df.T, str_df.T

In [ ]:
pop_df, str_df = create_table3(df)
str_df